In [2]:
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
from torchvision import datasets, transforms
import zipfile

In [4]:
def extract_data(_dir):
    for pic_type in ['color', 'grayscale', 'segmented']:
        with zipfile.ZipFile(f'{_dir}/{pic_type}.zip') as zf:
            zf.extractall(f'{_dir}/imgs/')

extract_data('data')

In [6]:
# Set up scaler.
scaler = StandardScaler()

def preprocess(data):
    if data is None:  # val/test can be empty
        return None
    # Read image files to pytorch dataset (only temporary).
    transform = transforms.Compose([
        transforms.Resize(64), 
        transforms.CenterCrop(64), 
        transforms.ToTensor()
    ])
    data = datasets.ImageFolder(data, transform=transform)

    # Convert to numpy arrays.
    images_shape = (len(data), *data[0][0].shape)
    images = np.zeros(images_shape)
    labels = np.zeros(len(data))
    for i, (image, label) in enumerate(data):
        images[i] = image
        labels[i] = label

    # Flatten.
    images = images.reshape(len(images), -1)

    # Scale to mean 0 and std 1.
    scaler.fit(images)
    images = scaler.transform(images)

    # Shuffle train set.
    images, labels = sklearn.utils.shuffle(images, labels)

    return [images, labels]

color_data = preprocess("data/imgs/color")
grayscale_data = preprocess("data/imgs/grayscale")
segmented_data = preprocess("data/imgs/segmented")

In [13]:
from sklearn.model_selection import train_test_split

CX_train, CX_test, Cy_train, Cy_test = train_test_split(color_data[0], color_data[1], test_size=0.2)
GX_train, GX_test, Gy_train, Gy_test = train_test_split(grayscale_data[0], grayscale_data[1], test_size=0.2)
SX_train, SX_test, Sy_train, Sy_test = train_test_split(segmented_data[0], segmented_data[1], test_size=0.2)

In [9]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [16]:
input_shape = CX_train[0].shape

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))